In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time_templates.utilities.fitting import plot_fit_curve, plot_fit_xy_xerr_yerr, bootstrap_fit, pretty_print_uncertaity
from time_templates.utilities.misc import histedges_equalN
import time_templates.utilities.plot as ttplt
from time_templates.utilities.stats import profile_1d
from time_templates.datareader.get_data import fetch_MC_data_from_tree, fetch_data
import scipy.stats as scistats
from time_templates.utilities.stats import weighted_corr
from time_templates.preprocessing.apply_cuts_df import apply_cuts_df
from time_templates.misc.Xmax import Fd_Xmax_resolution_at_lgE, plot_mean_Xmax_data, plot_sigma_Xmax_data, get_FD_Xmax_sys_uncertainty
from time_templates.misc.energy import SdlgE_resolution
from matplotlib.lines import Line2D
from uncertainties import ufloat, umath, unumpy
import uncertainties
from time_templates.misc import Xmax as Xmax_utils
from utils import *
from time_templates import data_path

SAVE = False
if SAVE:
    try:
        plt.style.use('thesis')
    except:
        pass
    nboot = 10000
else:
    %matplotlib notebook
    nboot = 100
label_mean_SD_Xmax = '$\\langle X_{\\rm max}(\\mathrm{SD}) \\rangle \, [\\rm g/cm^2]$'
lgEeV = "\\log_{{10}}{{\\left( E/\\mathrm{{eV}} \\right)}}"
FD_color = '#845B97'
def savefig(name, **kwargs):
    if SAVE:
        plt.savefig(f"/home/mart/auger/thesis/thesis_overleaf/figs/chapter6/{name}", bbox_inches='tight', **kwargs)

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'fitted', 'df_events_fitted_observer_icrc19_SD_SdlgE19_theta53_selected.csv'))
df.set_index('EventId', inplace=True)
cuts = {'Is6T5': True, 'SdCosTheta_new': (0.6, 1/1.1), 'Xmax_fit': (1, 3000), 'nstations_ttt_fit': (3, 1000), 'fit_success': 1, 'Sdr_new_min': (500, 1000)}
df = apply_cuts_df(df, cuts)
df['Xmax_fit_err'] = SD_Xmax_resolution(df['SdlgE'])
df['DXmax_fit'] = df['Xg'] - df['Xmax_fit']
df['DXmax_fit_corr'] = df.apply(lambda x: unbias_DXmax(x['DXmax_fit'], x['SdlgE']), axis=1)
df['DXmax_correction'] = df['DXmax_fit_corr'] - df['DXmax_fit']
df['Xmax_TTT'] = df['Xg'] - df['DXmax_fit_corr'] 
df['Xmax_fit_corr'] = df['Xmax_TTT'] + HYBRID_XMAX_TTT_OFFSET
df['SdSecTheta_new'] = 1/df['SdCosTheta_new']
df['DXmax_fit_corr'] = df['Xg'] - df['Xmax_fit_corr']

lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.5])
dlgE_bins = lgE_bins[1:] - lgE_bins[:-1]
lgE_bins_mean = (lgE_bins[1:] + lgE_bins[:-1])/2

In [ ]:
SCT_CUT = 1.25 #1/np.sqrt(0.8)
def sec_theta_correction(sec_theta, a, b=0, cutoff=SCT_CUT):
    return np.where(sec_theta < cutoff, a + b*(sec_theta - cutoff), a)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.625))

ax = ax1
ax, (x, y, yerr) = ttplt.plot_profile_1d(df['SdSecTheta_new'], df['Xmax_fit_corr'], bins=1/np.sqrt(np.linspace(1, 0.36, 22)), color='k', ax=ax)
mask = np.isfinite(x*y*yerr)
ax, (p, perr, chi2, ndf) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: sec_theta_correction(x, a, b, cutoff=1.2), p0=[760, 100], ax=ax, add_label=True,
               ebar_kws=dict(marker='', ls='', alpha=0), line_kws=dict(ls='--'), smoother_x=True)
print(p, perr, chi2, ndf)
ax, (p, perr, chi2, ndf) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: sec_theta_correction(x, a, b, cutoff=1.25), p0=[760, 100], ax=ax, add_label=True,
               ebar_kws=dict(marker='', ls='', alpha=0, color='k'), smoother_x=True)
print(p, perr, chi2, ndf)
ax, (p, perr, chi2, ndf) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: sec_theta_correction(x, a, b, cutoff=1.3), p0=[760, 100], ax=ax, add_label=True,
               ebar_kws=dict(marker='', ls='', alpha=0), smoother_x=True, line_kws=dict(ls='--'))
print(p, perr, chi2, ndf)
ax.set_xlabel('$\\sec{\\theta}$')
ax.set_ylabel(label_mean_SD_Xmax)
ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
new_tick_locations = np.array([1.1, 1.155, 1.31, 1.56])
ax_top.set_xticks(new_tick_locations)
new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
ax_top.tick_params(
    axis="x",
    which="minor",
    direction="in",
    top=False,
    labeltop=True,
    bottom=False,
    labelbottom=False,
)
ax_top.set_xlabel('$\\theta$')


ax = ax2

ax, (x, y, yerr) = ttplt.plot_profile_1d(df['SdSecTheta_new'], df['Xmax_fit_corr'], bins=1/np.sqrt(np.linspace(1, 0.36, 22)), stat=np.std, bootstraps=nboot, 
                     color='k', ax=ax)
ax.axvspan(1.09, 1.25, color='darkgrey', hatch='\\\\\\', fc='none', zorder=99, lw=0)
ax.set_xlim([1.09, 1.65])
# mask = np.isfinite(x*y*yerr) & (x > 1.25)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*(x-1.25), ax=ax, ebar_kws=dict(alpha=0, color='k'))
ax.set_xlabel('$\\sec{\\theta}$')
ax.set_ylabel("$\\sigma \\left( X_{\\rm max}(\\rm SD) \\right)\, [\\rm g/cm^2]$")
# ax.legend()

ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
new_tick_locations = np.array([1.1, 1.155, 1.31, 1.56])
ax_top.set_xticks(new_tick_locations)
new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
ax_top.tick_params(
    axis="x",
    which="minor",
    direction="in",
    top=False,
    labeltop=True,
    bottom=False,
    labelbottom=False,
)
ax_top.set_xlabel('$\\theta$')
f.subplots_adjust(wspace=0.3)
savefig('mean_sigma_Xmax_SD_vs_zenith.pdf')
# savefig('sigma_Xmax_SD_vs_zenith.pdf')
#chi2/ndf 9.4/8
# savefig('Xmax_SD_vs_zenith.pdf')

In [ ]:
#TODO: fix hardcoded
df['Xmax_fit_corr1'] = df['Xmax_fit_corr'] - sec_theta_correction(df['SdSecTheta_new'], 0, 209, 1.2)
df['Xmax_fit_corr2'] = df['Xmax_fit_corr'] - sec_theta_correction(df['SdSecTheta_new'], 0, 133, 1.25)
df['Xmax_fit_corr3'] = df['Xmax_fit_corr'] - sec_theta_correction(df['SdSecTheta_new'], 0, 94, 1.3)

In [ ]:
f, ax = plt.subplots(1)

ax, (x, y, yerr) = ttplt.plot_profile_1d(df['SdSecTheta_new'], df['Xmax_fit_corr'], bins=1/np.sqrt(np.linspace(1, 0.36, 22)), stat=np.std, bootstraps=nboot, 
                     color='k', ax=ax)
ax.axvspan(1.09, 1.25, color='darkgrey', hatch='\\\\\\', fc='none', zorder=99, lw=0)
ax.set_xlim([1.09, 1.65])
# mask = np.isfinite(x*y*yerr) & (x > 1.25)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*(x-1.25), ax=ax, ebar_kws=dict(alpha=0, color='k'))
ax.set_xlabel('$\\sec{\\theta}$')
ax.set_ylabel("$\\sigma \\left( X_{\\rm max}(\\rm SD) \\right)\, [\\rm g/cm^2]$")
# ax.legend()

ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
new_tick_locations = np.array([1.1, 1.155, 1.31, 1.56])
ax_top.set_xticks(new_tick_locations)
new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
ax_top.tick_params(
    axis="x",
    which="minor",
    direction="in",
    top=False,
    labeltop=True,
    bottom=False,
    labelbottom=False,
)
ax_top.set_xlabel('$\\theta$')

savefig('sigma_Xmax_SD_vs_zenith.pdf')
#chi2/ndf 9.4/8

In [ ]:
corrections = []
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.2])
for lgE_min, lgE_max in zip(lgE_bins[:-1], lgE_bins[1:]):
    DXmax = df.query(f'SdlgE >= {lgE_min} & SdlgE < {lgE_max}')['DXmax_fit'].mean()
#     print(DXmax)
    lgE = (lgE_min + lgE_max)/2
    corrections.append(-DXmax_bias(DXmax, ufloat(lgE, np.log10(1.14)), uncertainties.correlated_values(P_DXMAX_BIAS_CORR, COV_DXMAX_BIAS_CORR)))
    
f, ax = plt.subplots(1)
lgE = 0.5*(lgE_bins[1:] + lgE_bins[:-1])
ax.fill_between(10**lgE, [c.nominal_value-c.std_dev for c in corrections], [c.nominal_value+c.std_dev for c in corrections], color='k', hatch='\\\\\\', fc='none', lw=0)
# ax.plot(10**lgE, [c.nominal_value-c.std_dev for c in corrections], [c.nominal_value+c.std_dev for c in corrections], color='k', hatch='\\\\\\', fc='none', lw=0)

ax.set_ylabel('Average $X_{\\rm max}$ bias correction $[\\rm g/cm^2]$')
ax.set_xscale('log')
ax.set_xlabel('E [eV]')
ax.axhline(0, ls=':', color='k', lw=0.7)
ax.set_ylim([-12, 12])

ax_top = ax.twiny()
ax_top.set_xlim(np.log10(ax.get_xlim()))
new_tick_locations = np.array([19, 19.5, 20])
ax_top.set_xticks(new_tick_locations)
ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)

savefig('ave_Xmax_correction_MC_bias.pdf')

In [ ]:
def calc_sys_Xmax_TTT(lgE, ax=None):
    SYS_AGE = 3
    SYS_ZENITH = 4 #average deviation (std_dev) between vertical and inclined events + 1 from uncertainty
    SYS_PRIM = 2
    SYS_HYBRID_CALLIBRATION = 2.8
#     SYS_ZENITH = 3# mean of deviation from propagation of zenith angle correction
    #This is already conservatively estimated, setting the uncertainty on the slope parameter to 10 and the norm parameter error to 2
    #and rounding up
#     lgE_err = np.log10(1.14) #SdlgE_resolution(lgE)
    SYS_DXMAX_BIAS_CORR = 1.5 - 0. * (lgE-19.5) #conservative, flat in energy for DX in this data set

#     Xmax_bias_corr_sys = unumpy.std_devs(DXmax_bias_func(df['DXmax_fit'].values,
#                                                             unumpy.uarray(df['SdlgE'].values, lgE_err),
#                                                             uncertainties.correlated_values(p_DXmax_bias_corr, p_DXmax_bias_corr_cov)))


    SD_low = np.sqrt(SYS_ZENITH**2 + SYS_AGE**2)
    SD_up = np.sqrt(SYS_ZENITH**2 + SYS_AGE**2)
    MC = np.sqrt(SYS_DXMAX_BIAS_CORR**2 + SYS_PRIM**2)
    fd_sys_low, fd_sys_up = get_FD_Xmax_sys_uncertainty(lgE)
    fd_sys_low = np.sqrt(fd_sys_low**2 + SYS_HYBRID_CALLIBRATION**2)
    fd_sys_up = np.sqrt(fd_sys_up**2 + SYS_HYBRID_CALLIBRATION**2)

#     total_low = (Xmax_bias_corr_sys_prim**2 + fd_sys_low**2 + other**2)**0.5
#     total_up = (Xmax_bias_corr_sys_prim**2 + fd_sys_up**2 + other**2)**0.5
    total_low = (SYS_DXMAX_BIAS_CORR**2 + SYS_PRIM**2 + SYS_HYBRID_CALLIBRATION**2 + SYS_AGE**2 + SYS_ZENITH**2 + fd_sys_low**2)**0.5
    total_up = (SYS_DXMAX_BIAS_CORR**2 + SYS_PRIM**2 + SYS_HYBRID_CALLIBRATION**2 + SYS_AGE**2 + SYS_ZENITH**2 + fd_sys_up**2)**0.5

    if ax is not None:
#         f, ax = plt.subplots(1)
        E = 10**lgE
        ax.plot(E, -fd_sys_low, '--', color=FD_color, label='FD + calibr.')
        ax.plot(E, fd_sys_up, '--', color=FD_color)
        ax.plot(E, MC, color='r', ls='-.', label='MC bias')
        ax.plot(E, -MC, color='r', ls='-.')
        ax.axhline(SD_up, color='g', ls=':', label='zenith + age')
        ax.axhline(-SD_low, color='g', ls=':')
        ax.plot(E, -total_low, 'k-', label='total')
        ax.plot(E, total_up, 'k-')
        ax.legend(ncol=2, loc=1)
        ax.set_ylabel('$\\langle X_{\\rm max}(\\mathrm{SD}) \\rangle $ systematics [$\\rm g/cm^2$]')
        ax.set_xlabel('$E$ [eV]')
        ax.set_ylim([-35, 35])
        ax.set_xscale('log')
        ax.set_xlim([1e19, 10**lgE.max()])
        
        ax_top = ax.twiny()
        ax_top.set_xlim(np.log10(ax.get_xlim()))
        new_tick_locations = np.array([19, 19.5, 20])
        ax_top.set_xticks(new_tick_locations)
        ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)  
#         ax.axhline(0, ls='--', color='k', lw=0.7)
    return total_low, total_up

def calc_sys_sigma_Xmax_TTT(lgE, ax=None):
    lgE = unumpy.uarray(lgE, np.log10(1.14))
    #important to include correlation (is almost ~1)
    a, b = uncertainties.correlated_values(P_HYBRID_XMAX_RESOLUTION, COV_HYBRID_XMAX_RESOLUTION)
    total_resolution = resolution_func(lgE, a, b)
    FD = Fd_Xmax_resolution_at_lgE(unumpy.nominal_values(lgE))
    FD_resolution_sys = 2 #above 10^19 eV seems almost constant
    SD = (total_resolution**2 - unumpy.uarray(FD, FD_resolution_sys)**2)**0.5
    sys_resolution = unumpy.std_devs(SD)
    MC_bias_correction = 2
    zenith_diff = 3
    total = (MC_bias_correction**2 + zenith_diff**2 + sys_resolution**2)**0.5
    
    if ax is not None:
#         f, ax = plt.subplots(1)
        E = 10**unumpy.nominal_values(lgE)
        ax.axhline(MC_bias_correction, color='r', ls='-.', label='MC bias')
        ax.axhline(-MC_bias_correction, color='r', ls='-.')
        ax.plot(E, sys_resolution, '--', color=FD_color, label='resolution')
        ax.plot(E, total, 'k-', label='total')
        if zenith_diff > 0:
            ax.axhline(zenith_diff, color='g', ls=':', label='zenith angle')
            ax.axhline(-zenith_diff, color='g', ls=':')
        ax.plot(E, -sys_resolution, '--', color=FD_color)
        ax.plot(E, -total, 'k-')
        ax.legend()
        ax.legend(ncol=2)
        ax.set_ylabel('$\\sigma \\left( X_{\\rm max}(\\mathrm{SD}) \\right) $ systematics [$\\rm g/cm^2$]')
        ax.set_xlabel('$E$ [eV]')
        ax.set_ylim([-20, 20])
        ax.set_xscale('log')
        ax.set_xlim([1e19, E.max()])
        
        ax_top = ax.twiny()
        ax_top.set_xlim(np.log10(ax.get_xlim()))
        new_tick_locations = np.array([19, 19.5, 20])
        ax_top.set_xticks(new_tick_locations)
        ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)
    return total

f, ax = plt.subplots(1)
calc_sys_Xmax_TTT(np.linspace(19, 20.1), ax=ax)
savefig('mean_Xmax_SD_systematics_lgE.pdf')
f, ax = plt.subplots(1)

calc_sys_sigma_Xmax_TTT(np.linspace(19, 20.1), ax=ax)
savefig('sigma_Xmax_SD_systematics_lgE.pdf')

In [ ]:
def plot_mean_Xmax_SD(ax, df, lgE_bins, key='Xmax_fit_corr2', lgE=False, plot_sys=True, hatch='\\\\\\', plot_n_entries=False, **kwargs):
    import matplotlib.transforms as transforms
    

    if lgE:
        x, y, yerr, n = profile_1d(df['SdlgE'], df[key], bins=lgE_bins)
    else:
        x, y, yerr, n = profile_1d(10**df['SdlgE'], df[key], bins=10**lgE_bins, bootstraps=nboot)
            
    if lgE:
        lgx = x
    else:
        lgx = np.log10(x)
    std = np.maximum(yerr*np.sqrt(n), SD_Xmax_resolution(lgx))
    yerr = std/np.sqrt(n)

    ax.errorbar(x, y, yerr=yerr, ls='', **kwargs)
    sys_low = 0
    sys_up = 0
    if plot_sys:
#         x = np.insert(x, 0, 1e19)
#         x = np.append(x, 10**20.1)
#         y = np.insert(y, 0, y[0])
#         y = np.append(y, y[-1])
        sys_low, sys_up = calc_sys_Xmax_TTT(np.log10(x))
        ttplt.plot_sys_brackets(x, y, sys_low, sys_up, ax, color=kwargs['color'])
#         ax.fill_between(x, y-sys_low, y+sys_up, hatch=hatch, color='darkgrey', facecolor='none', lw=0, zorder=1)
    if plot_n_entries:
        trans = transforms.blended_transform_factory(x_transform=ax.transData, y_transform=ax.transAxes)
        for i, _n in enumerate(n):
            ax.annotate(int(_n), xy=(x[i], 0), xycoords=trans, fontsize=8,
                ha="center", va="bottom", xytext=(0, 1), textcoords="offset pixels")
    return x, y, yerr, sys_low, sys_up, n

In [ ]:
f, ax = plt.subplots(1, figsize=(4.4, 3.6))
# lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.85, 19.9, 19.95, 20, 20.05, 20.1, 21])
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 22])
df_ = df#.query('SdSecTheta > 1 & DXmax_fit_corr2 > 50')

Emean, Xmax_mean, Xmax_err, Xmax_sys_low, Xmax_sys_up, nevents = plot_mean_Xmax_SD(ax, df_, lgE_bins, key='Xmax_fit_corr2', marker='o', color='k', label='SD', zorder=99, ms=5)

# plot_mean_Xmax_SD(ax, df_, lgE_bins, key='Xmax_fit_corr1', marker='x', color='r', label='SD', zorder=99, ms=5, plot_sys=False)
# plot_mean_Xmax_SD(ax, df_, lgE_bins, key='Xmax_fit_corr', marker='v', color='g', label='SD', zorder=99, ms=5, plot_sys=False)

plot_mean_Xmax_data(ax, lgEmin=19, lgEmax=20.2, data_color=FD_color, data_kwargs=dict(alpha=0.9), data_marker='s', y_offset=0, plot_sys=True)

# lgE_bins_FD = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 22])
# ttplt.plot_profile_1d(10**df_FD['SdlgE'], df_FD['FdXmax'], bins=10**lgE_bins_FD, marker='v', color=FD_color, ax=ax, plot_n_entries=False)
# ttplt.plot_profile_1d(10**df_FD['SdlgE'], df_FD['Xmax_fit_corr'], bins=10**lgE_bins_FD, marker='x', color='g', ax=ax, plot_n_entries=False)
# df_FD_icrc19 = fetch_data(key="FD_Events_icrc19")
# df_FD_icrc19 = df_FD_icrc19.groupby('EventId').mean()
# df_FD_icrc19['FdXmax_acceptance'] = Xmax_utils.Fd_Xmax_acceptance(df_FD_icrc19['FdXmax'], 10**df_FD_icrc19['FdlgE'])
# ttplt.plot_profile_1d(10**df_FD_icrc19['FdlgE'], df_FD_icrc19['FdXmax'], bins=10**lgE_bins_FD, weights=1/df_FD_icrc19['FdXmax_acceptance'],
#                       marker='x', ax=ax)

ax.set_xlim([10**19.0, 10**20.2])
ax.set_ylim([690, 880])
err_SD = ax.errorbar([0], [0], yerr=[1], color='k', ls='', marker='o')
err_FD = ax.errorbar([0], [0], yerr=[1], color=FD_color, ls='', marker='s')
handles = [err_SD, err_FD]
labels = ['SD', 'FD']
leg1 = ax.legend(handles, labels, ncol=1, frameon=False, loc=2)
handles = [Line2D([0], [0], color='grey', ls='-'),
           Line2D([0], [0], color='grey', ls='--'),
           Line2D([0], [0], color='grey', ls=':')]
labels = ['EPOS-LHC', 'QGSJet-II.04', 'Sibyll2.3d']
leg2 = ax.legend(handles, labels, ncol=1, frameon=False, loc=4)
ax.add_artist(leg1)

ax.annotate('proton', color='b', xy=(1.1e19, 54), rotation=0)
ax.annotate('iron', color='r', xy=(1.1e19, 19), rotation=0)

ax.set_ylabel('$\\langle X_{\\rm max}\\rangle \\, [\\rm g/cm^2]$')
ax.annotate('proton', color='b', xy=(1.1e19, 797), rotation=13)
ax.annotate('iron', color='r', xy=(1.1e19, 703), rotation=13)
# ax.set_ylim([750, 800])

ax_top = ax.twiny()
ax_top.set_xlim(np.log10(ax.get_xlim()))
new_tick_locations = np.array([19, 19.5, 20])
ax_top.set_xticks(new_tick_locations)
ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)

savefig('SD_mean_Xmax_vs_lgE.pdf')

In [ ]:
list(df.keys())

In [ ]:
df_sel = df[['Xmax_TTT', 'Xmax_fit_corr2', 'SdDecl', 'SdRA', 'SdlgE', 'year', 'Xg', 'Xmax_fit_err', 'SdCosTheta', 'SdAzimuth', 'GPSSecond']]
df_sel = df_sel.rename(columns={'Xmax_fit_corr2': 'Xmax', 'Xmax_TTT': 'Xmax_TTT_no_corr', 'Xmax_fit_err': 'Xmax_err'})
df_sel.to_csv("/home/mart/auger/data/time_templates/SD_Xmax_events.csv")

In [ ]:
df_mean = pd.DataFrame({'lgE_mean': np.log10(Emean), 'lgE_low': lgE_bins[:-1], 'lgE_up': lgE_bins[1:], 'Xmax_mean': Xmax_mean, 'stat_err': Xmax_err,
                        'sys_low': Xmax_sys_low, 'sys_up': Xmax_sys_up, 'n_events': nevents})
df_mean['n_events'] = df_mean['n_events'].astype('int')
# df_mean.to_csv("/home/mart/auger/projects/time_templates/SD_Xmax_mean.txt",
#     formatters={'lgE_mean': '${:.2f}'.format, 'Xmax_mean': '${:.0f}'.format, 'stat_err': '${:.0f}'.format, 'sys_low': '${:.0f}'.format, 'sys_high': '${:.0f}'.format})
str_df = df_mean.to_string(index=False, justify='center', formatters={'lgE_mean': '{:.2f}'.format, 'Xmax_mean': '{:.1f}'.format, 'stat_err': '{:.1f}'.format, 
                                       'sys_low': '{:.1f}'.format, 'sys_up': '{:.1f}'.format})
with open('/home/mart/auger/projects/time_templates/SD_Xmax_mean.txt', 'w') as out:
    out.write(str_df)

In [ ]:
f, ax = plt.subplots(1, figsize=(4.4, 3.6))
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 22])

plot_sigma_Xmax_data(ax, lgEmin=19, lgEmax=20.2, data_color=FD_color, data_kwargs=dict(alpha=0.9), more_MC_lines=False)

def plot_sigma_Xmax(df_, ax, lgE_bins=lgE_bins, key='Xmax_fit_corr2', color='k', marker='o', plot_sys=True):

#     ax, (x, y, yerr) = ttplt.plot_profile_1d(10**df_['SdlgE'], df_[key], bins=10**lgE_bins, stat=np.std, bootstraps=nboot,
#                               marker='.', ax=ax, color='g', plot_n_entries=False, alpha=0.)
    x, y, yerr, n = profile_1d(10**df_['SdlgE'], df_[key],  bins=10**lgE_bins, stat=np.std, bootstraps=nboot)

#     print(y[0], x[0])
    y = np.sqrt(y**2 - SD_Xmax_resolution(np.log10(x))**2)
    ax.errorbar(x, y, yerr=yerr, marker=marker, color=color, ls='', label='subtracted resolution', ms=5)
#     x = np.insert(x, 0, 1e19)
#     x = np.append(x, 10**20.1)
    sys = calc_sys_sigma_Xmax_TTT(np.log10(x))
#     y = np.insert(y, 0, y[0])
#     y = np.append(y, y[-1])
    sys_low = sys
    sys_up = sys
    if plot_sys:
        ttplt.plot_sys_brackets(x, y, sys_low, sys_up, ax=ax, color='k')
#     ax.fill_between(x, y-sys, y+sys, hatch='\\\\\\', color='darkgrey', facecolor='none', lw=0, zorder=1)
    return x, y, yerr, sys_low, sys_up, n
    
    
ax.set_ylabel("$\\sigma \\left( X_{\\rm max} \\right)\, [\\rm g/cm^2]$")
ax.set_xlabel('$E$ [eV]')

err_SD = ax.errorbar([0], [0], yerr=[1], color='k', ls='', marker='o')
err_FD = ax.errorbar([0], [0], yerr=[1], color=FD_color, ls='', marker='s')

handles = [err_SD, err_FD]
labels = ['SD', 'FD']
leg1 = ax.legend(handles, labels, ncol=1, frameon=False, loc=2)
handles = [Line2D([0], [0], color='grey', ls='-'),
           Line2D([0], [0], color='grey', ls='--'),
           Line2D([0], [0], color='grey', ls=':')]
labels = ['EPOS-LHC', 'QGSJet-II.04', 'Sibyll2.3d']
leg2 = ax.legend(handles, labels, ncol=1, frameon=False, loc=1)
ax.add_artist(leg1)

ax.annotate('proton', color='b', xy=(1.2e19, 53.5), rotation=0)
ax.annotate('iron', color='r', xy=(1.2e19, 18.5), rotation=0)

ax.set_xlim([10**19, 10**20.2]);
ax.set_ylim([0, 80])
    
# plot_sigma_Xmax(df, ax=ax, color='m')
Emean, Xmax_sigma, Xmax_sigma_err, Xmax_sigma_sys_low, Xmax_sigma_sys_up, n_events = plot_sigma_Xmax(df.query('SdSecTheta_new > 1.25'), ax=ax, color='k')
# plot_sigma_Xmax(df.query('SdSecTheta_new > 1.25'), ax=ax, color='c', kde=False, plot_sys=False)

# plot_sigma_Xmax(df.query('SdSecTheta_new > 1.25 & SdSecTheta_new < 1.4'), ax=ax, color='orange')
ax_top = ax.twiny()
ax_top.set_xlim(np.log10(ax.get_xlim()))
new_tick_locations = np.array([19, 19.5, 20])
ax_top.set_xticks(new_tick_locations)
ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)
savefig('SD_sigma_Xmax_vs_lgE.pdf')

In [ ]:
f, axes = plt.subplots(5, 2, figsize=(6, 10.1))
axes = axes.flatten()
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 21])
bins = np.arange(df['Xmax_fit_corr2'].min(), df['Xmax_fit_corr2'].max(), 20)

for i, lgE_min in enumerate(lgE_bins[:-1]): 
    ax = axes[i]

    df_ = df.query(f'SdlgE > {lgE_min} & SdlgE < {lgE_bins[i+1]}')
    if i == len(lgE_bins) - 2:
        lgE_max = round(df_['SdlgE'].max(), 2)+0.01
    else:
        lgE_max = lgE_bins[i+1]
    ax.annotate(f'${lgE_min} \\leq \\log_{{10}}(E/\\rm eV) < {lgE_max}$', xy=(0.05, 0.95), xycoords='axes fraction', va='top', ha='left', fontsize=10)
    ax, _ = ttplt.plot_hist(df_['Xmax_fit_corr2'], bins=bins, histtype='step', ax=ax, color='k', bootstraps=0)
#     kde = scistats.gaussian_kde(df_['Xmax_fit_corr2'])
#     x = np.linspace(bins[0], bins[-1], 100)
#     ax.plot(x, kde.pdf(x)*len(df_)*20, 'k--')
    handles, labels = ax.get_legend_handles_labels()
    handles = [Line2D([0], [0], ls='', marker='') for h in handles]
    ax.legend(handles, labels, loc=5, fontsize=10, frameon=False)
    ax.set_xlabel('$X_{\\rm max}(\\rm SD)\, [\\rm g/cm^2]$', labelpad=-1)
    ax.set_ylabel('\# events')
#     ax.legend(loc=4, fontsize=8)
    ymin, ymax = ax.get_ylim()
    ax.set_ylim([ymin, ymax*1.25])
f.subplots_adjust(wspace=0.28, hspace=0.33)
savefig('SD_Xmax_hist_lgE_bins.pdf')

In [ ]:
year_bins = np.arange(5.5, 18.5, 1) 
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6, 6))
ax = ax1
ax, (x, y, yerr) = ttplt.plot_profile_1d(df['year']-2000, df['Xmax_fit_corr2'], bins=year_bins, ax=ax, bootstraps=0, color='k')
mask = np.isfinite(x*y*yerr)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', marker='s', ls=''))
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*x, ax=ax, ebar_kws=dict(color='k', alpha=0, ls=''))

ax.set_xticks(np.arange(5, 19, 5))
ax.set_xlabel('year since 2000')
# ax.legend(loc=1)
# ax.grid()
ax.set_ylabel("$\\langle X_{\\rm max}(\\mathrm{SD}) \\rangle \\, [\\rm g/cm^2]$")
ax.set_ylim([762, 788])
ax = ax2
AOP_bins = np.linspace(2.8, 3.4, 11)
ax, (x, y, yerr) = ttplt.plot_profile_1d(df['WCDCharge']/df['WCDPeak'], df['Xmax_fit_corr2'], bins=AOP_bins, ax=ax, bootstraps=0, color='k')
mask = np.isfinite(x*y*yerr)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', marker='s', ls=''))

ax.set_ylabel("$\\langle X_{\\rm max}(\\mathrm{SD}) \\rangle \\, [\\rm g/cm^2]$")
ax.set_xlabel('$\\langle \\rm AOP\\rangle$')
# ax.set_ylim([762, 788])
# ax.legend(loc=2)
# ax.set_ylim([-50, 40])

df_ = df.query('SdSecTheta_new > 1.25')
ax = ax3
ax, (x, y, yerr) = ttplt.plot_profile_1d(df_['year']-2000, df_['Xmax_fit_corr2'], bins=year_bins, ax=ax, bootstraps=nboot, color='k', stat=np.std)
mask = np.isfinite(x*y*yerr)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', marker='s', ls=''))
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*x, ax=ax, ebar_kws=dict(color='k', alpha=0, ls=''))

ax.set_xticks(np.arange(5, 19, 5))
ax.set_xlabel('year since 2000')
ax.set_ylim([35, 65])
ax.set_ylabel("$\\sigma \\left( X_{\\rm max}(\\rm SD) \\right)\, [\\rm g/cm^2]$")

ax = ax4
ax, (x, y, yerr) = ttplt.plot_profile_1d(df['WCDCharge']/df['WCDPeak'], df['Xmax_fit_corr2'], bins=AOP_bins, ax=ax, bootstraps=nboot, color='k', stat=np.std)
mask = np.isfinite(x*y*yerr)
# plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', marker='s', ls=''))

ax.set_ylabel("$\\sigma \\left( X_{\\rm max}(\\rm SD) \\right)\, [\\rm g/cm^2]$")
ax.set_xlabel('$\\langle \\rm AOP\\rangle$')
ax.set_ylim([35, 65])

f.subplots_adjust(wspace=0.35)
savefig('SD_Xmax_bias_year_AOP.pdf')

In [ ]:
df['Xvg'] = df['Xg'] * df['SdCosTheta_new']
df['SdAzimuth_deg'] = np.rad2deg(df['SdAzimuth'])
# df['SdAzimuth_new_deg'] = np.rad2deg(df['SdAzimuth_new'])
df['SdCosPhi'] = np.cos(df['SdAzimuth'])
# df['SdCosPhi_new'] = np.cos(df['SdAzimuth_new'])

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(6, 9))
axes = axes.flatten()

keys = ['month', 'hour', 'Pressure', 'Temperature', 'Xvg', 'SdAzimuth_deg']

binss = [np.arange(0.1, 13, 1), np.arange(0.1, 25, 2), np.linspace(0.81, 0.87, 13), np.linspace(260, 305, 12), np.linspace(860, 895, 12), np.arange(0, 370, 30)]

for key, bins, ax in zip(keys, binss, axes):
    ax, (x, y, yerr) = ttplt.plot_profile_1d(df[key], df['Xmax_fit_corr2'], bins=bins, ax=ax, color='k')
#     mask = np.isfinite(x*y*yerr)
#     xmean = 0#np.mean(x[mask])
#     xstd = 1#np.std(x[mask])
#     plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', ls=''))
#     plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-xmean)/xstd, ax=ax, ebar_kws=dict(color='k', ls=''), line_kws=dict(ls='--'))

    ax.set_ylabel(label_mean_SD_Xmax)
    ax.set_ylim([750, 800])
#     handles, labels = ax.get_legend_handles_labels()
#     leg1 = ax.legend([handles[0]],[labels[0]], loc=2)
#     leg2 = ax.legend([handles[1]], [labels[1]], loc=4)
#     ax.add_artist(leg1)
    
axes[0].set_xlabel('month')
axes[0].set_xticks([0, 2, 4, 6, 8, 10, 12])
axes[1].set_xticks([0, 4, 8, 12, 16, 20, 24])
axes[1].set_xlabel("hour")
axes[2].set_xticks([0.82, 0.83, 0.84, 0.85, 0.86, 0.87])
axes[2].set_xlabel('pressure [bar]')
axes[3].set_xlabel('Temperature [K]')
axes[3].set_xticks([260, 270, 280, 290, 300])
axes[4].set_xlabel('vertical slant depth [$\\rm g/cm^2$]')
# axes[5].set_xticks([-180, -90, 0, 90, 180])
axes[5].set_xticks([0, 90, 180, 270, 360])

axes[5].set_xlabel('Azimuth [deg]')
f.subplots_adjust(wspace=0.3, hspace=0.25)
savefig('mean_Xmax_further_syst.pdf')

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(6, 9))
axes = axes.flatten()

keys = ['month', 'hour', 'Pressure', 'Temperature', 'Xvg', 'SdAzimuth_deg']

binss = [np.arange(0.1, 13, 1), np.arange(0.1, 25, 2), np.linspace(0.81, 0.87, 13), np.linspace(260, 305, 12), np.linspace(860, 895, 12), np.arange(-180, 190, 30)]

df_ = df.query('SdSecTheta_new > 1.25')
for key, bins, ax in zip(keys, binss, axes):
    ax, (x, y, yerr) = ttplt.plot_profile_1d(df_[key], df_['Xmax_fit_corr2'], bins=bins, ax=ax, color='k', stat=np.std, bootstraps=nboot)
#     mask = np.isfinite(x*y*yerr)
#     xmean = 0#np.mean(x[mask])
#     xstd = 1#np.std(x[mask])
#     plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: a+0*x, ax=ax, ebar_kws=dict(color='k', ls=''))
#     plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-xmean)/xstd, ax=ax, ebar_kws=dict(color='k', ls=''), line_kws=dict(ls='--'))
    ax.set_ylabel("$\\sigma \\left( X_{\\rm max}(\\rm SD) \\right)\, [\\rm g/cm^2]$")
#     handles, labels = ax.get_legend_handles_labels()
#     leg1 = ax.legend([handles[0]],[labels[0]], loc=2)
#     leg2 = ax.legend([handles[1]], [labels[1]], loc=4)
#     ax.add_artist(leg1)
    ax.set_ylim([20, 70])
    
axes[0].set_xlabel('month')
axes[0].set_xticks([0, 2, 4, 6, 8, 10, 12])
axes[1].set_xticks([0, 4, 8, 12, 16, 20, 24])
axes[1].set_xlabel("hour")
axes[2].set_xticks([0.82, 0.83, 0.84, 0.85, 0.86, 0.87])
axes[2].set_xlabel('pressure [bar]')
axes[3].set_xlabel('Temperature [K]')
axes[3].set_xticks([260, 270, 280, 290, 300])
axes[4].set_xlabel('vertical slant depth [$\\rm g/cm^2$]')
# axes[5].set_xticks([-180, -90, 0, 90, 180])
axes[5].set_xticks([0, 90, 180, 270, 360])
axes[5].set_xlabel('Azimuth [deg]')
f.subplots_adjust(wspace=0.3, hspace=0.25)
savefig('sigma_Xmax_further_syst.pdf')

In [ ]:
plt.style.use('thesis')

f, ax = plt.subplots(1, figsize=(4.4, 3.6))
# lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.85, 19.9, 19.95, 20, 20.05, 20.1, 21])
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 22])
df_ = df.query('SdSecTheta > 1.')
plot_mean_Xmax_SD(ax, df_, lgE_bins, marker='o', color='k', label='SD', zorder=99, ms=5)

plot_mean_Xmax_data(ax, lgEmin=19, lgEmax=20.2, data_color=FD_color, data_kwargs=dict(alpha=0.), plot_sys=False)

ax.set_xlim([10**19.0, 10**20.2])
ax.set_ylim([690, 880])

err_SD = ax.errorbar([0], [0], yerr=[1], color='k', ls='', marker='o')
err_Delta = ax.errorbar([0], [0], yerr=[1], color='c', ls='', marker='>')
err_Delta2 = ax.errorbar([0], [0], yerr=[1], color='g', ls='', marker='<')

handles = [err_SD, err_Delta, err_Delta2]
labels = ['SD: TTT (this work)', 'SD: Delta (ICRC19)', 'SD: Delta (PRD17)']

leg1 = ax.legend(handles, labels, ncol=1, frameon=False, loc=2)

handles = [Line2D([0], [0], color='grey', ls='-'),
           Line2D([0], [0], color='grey', ls='--'),
           Line2D([0], [0], color='grey', ls=':')]
labels = ['EPOS-LHC', 'QGSJet-II.04', 'Sibyll2.3d']
leg2 = ax.legend(handles, labels, ncol=1, frameon=False, loc=4)
ax.add_artist(leg1)

ax.annotate('proton', color='b', xy=(1.1e19, 54), rotation=0)
ax.annotate('iron', color='r', xy=(1.1e19, 19), rotation=0)
ax.set_ylabel('$\\langle X_{\\rm max}\\rangle \\, [\\rm g/cm^2]$')

lgE_Delta, Xmax_Delta, Xmax_Delta_err = np.genfromtxt('/home/mart/auger/projects/time_templates/time_templates/misc/XmaxSD_1500m_60deg_2018_08.txt').T
ax.errorbar(10**lgE_Delta, Xmax_Delta, yerr=Xmax_Delta_err, marker='>', ls='', color='c', ms=5)

lgE_Delta, Xmax_Delta, Xmax_Delta_err = np.genfromtxt('/home/mart/auger/projects/time_templates/time_templates/misc/DeltaXmax_PRD.txt').T
ax.errorbar(10**lgE_Delta, Xmax_Delta, yerr=Xmax_Delta_err, marker='<', ls='', color='g', ms=5)
mask = lgE_Delta > 19
ttplt.plot_sys_brackets(10**lgE_Delta[mask], Xmax_Delta[mask], ylow=11*np.ones_like(lgE_Delta[mask]), yup=11*np.ones_like(lgE_Delta[mask]), color='g', ax=ax)


ax_top = ax.twiny()
ax_top.set_xlim(np.log10(ax.get_xlim()))
new_tick_locations = np.array([19, 19.5, 20])
ax_top.set_xticks(new_tick_locations)
ax_top.set_xlabel('$\\log_{10}\\left(E/\\mathrm{eV} \\right)$', labelpad=5)

plt.savefig('Delta_compare.pdf',bbox_inches='tight')
# savefig('Xmax_SD_Delta_compare.pdf')

In [ ]:
df_ = df.query('SdSecTheta > 1.')
lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 21])

def lin_func(lgE, a=779.5, b=26.5):
    return a + b * (lgE - 19)

def piece_wise_func(lgE, a, b, d, c=19.66):
     return np.where(x < c, a+b*(x-c), a+d*(x-c))

p0_1 = [777, 27]
p0_2 = [777, 27, 27]
    
f, ax = plt.subplots(1)
x, y, yerr, sys_low, sys_up, n = plot_mean_Xmax_SD(ax, df_, lgE_bins, marker='o', color='k', plot_sys=False, lgE=True, plot_n_entries=False);

mask = np.isfinite(x*y*yerr)

ax, (p, perr, chi2_0, ndf_0) = plot_fit_curve(x, y, yerr=yerr, func=lin_func, p0=p0_1, ax=ax,
                                              ebar_kws=dict(alpha=0, color='k'), param_names=['C', 'D_{10}'])

# ax, (p, perr, chi2_1, ndf_1) = plot_fit_curve(x, y, yerr=yerr, func=piece_wise_func, p0=p0_2, ax=ax,
#                                               ebar_kws=dict(alpha=0, color='#0C5DA5'), param_names=['a_0', '\\gamma_1', '\gamma_2', '\\mathrm{lg}E_c'],
#                                               bounds=[(700, 1, 1), (800, 200, 200)])

# TS_fit = (chi2_0-chi2_1)/(ndf_0-ndf_1)

# print(TS_fit)
ax.legend(loc=2)
# ax.set_ylim([680, 880])
# ax.grid()
ax.set_xlabel('$\\log_{10}\\left(E/\\rm eV \\right)$')
ax.set_ylabel('$\\langle X_{\\rm max}\\rangle \\, [\\rm g/cm^2]$')
savefig('SD_Xmax_mean_elongation_rate.pdf')

In [ ]:
from time_templates.misc.Xmax import lnA_from_Xmax, get_FD_Xmax
f, axes = plt.subplots(1, 3, figsize=(6, 2.5), sharey=True)

lgE_bins = np.array([19, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 22])
df_ = df.query('SdSecTheta > 1')
x, y, yerr, sys_low, sys_up, n = plot_mean_Xmax_SD(axes[0], df_, lgE_bins, marker='o', color='none', plot_sys=False);

y = unumpy.uarray(y, yerr)
xsys = x.copy()
# xsys = np.insert(xsys, 0, 1e19)
# xsys = np.append(xsys, 10**20.1)
ysys = unumpy.nominal_values(y).copy()
# ysys = np.insert(ysys, 0, ysys[0])
# ysys = np.append(ysys, ysys[-1])
sys_low, sys_up = calc_sys_Xmax_TTT(np.log10(xsys))

FD_Xmax, FD_Xmax_err, FD_Xmax_low, FD_Xmax_up, lgE_FD = np.array(get_FD_Xmax(19, 22))
for HIM, ax in zip(['EPOS-LHC', 'QGSJet-II.04', 'Sibyll2.3d'], axes):
    #SD
    lnA = lnA_from_Xmax(y, x, HIM)
#     print(lnA)
    lnA_sys_up = np.array(lnA_from_Xmax(ysys-sys_low, xsys, HIM))
    lnA_sys_low = np.array(lnA_from_Xmax(ysys+sys_up, xsys, HIM))
    ax.errorbar(x, unumpy.nominal_values(lnA), yerr=unumpy.std_devs(lnA), color='k', marker='o', ls='', zorder=99)
    ttplt.plot_sys_brackets(xsys, unumpy.nominal_values(lnA), unumpy.nominal_values(lnA)-lnA_sys_low, lnA_sys_up - unumpy.nominal_values(lnA), ax, color='k', size=8)

#     ax.fill_between(xsys, lnA_sys_low, lnA_sys_up, hatch='\\\\\\', color='darkgrey', facecolor='none', lw=0, zorder=1)
    
    #FD
#     FD_lnA = lnA_from_Xmax(unumpy.uarray(FD_Xmax, FD_Xmax_err), 10**lgE_FD, HIM)
#     ax.errorbar(10**lgE_FD, unumpy.nominal_values(FD_lnA), yerr=unumpy.std_devs(FD_lnA), color=FD_color, marker='s', ls='', zorder=98, alpha=0.9)
#     FD_lnA_sys_low = lnA_from_Xmax(FD_Xmax-FD_Xmax_low, 10**lgE_FD, HIM) - unumpy.nominal_values(FD_lnA)
#     FD_lnA_sys_up = unumpy.nominal_values(FD_lnA) - lnA_from_Xmax(FD_Xmax+FD_Xmax_up, 10**lgE_FD, HIM) 
#     ttplt.plot_sys_brackets(10**lgE_FD, unumpy.nominal_values(FD_lnA), FD_lnA_sys_low, FD_lnA_sys_up, ax, color=FD_color, size=10)
    ax.axhline(0, ls=':', color='k')
    ax.axhline(np.log(4), ls=':', color='k')
    ax.axhline(np.log(14), ls=':', color='k')
    ax.axhline(np.log(55.8), ls=':', color='k')
    ax.set_ylim([-0.3, 4.5])
    ax.set_xscale('log')
    ax.set_title(HIM)
    ax.set_xlabel('$E$ [eV]')
    ax.set_xlim([10**18.9, 10**20.15]);

x = 1.5e20
axes[0].set_ylabel('$\\langle \\ln{A} \\rangle$')

axes[2].annotate('Fe', xy=(x, np.log(55.8)-0.1), annotation_clip=False)
axes[2].annotate('N', xy=(x, np.log(14)-0.1), annotation_clip=False)
axes[2].annotate('He', xy=(x, np.log(4)-0.1), annotation_clip=False)
axes[2].annotate('p', xy=(x, -0.1), annotation_clip=False)
f.subplots_adjust(wspace=0.)

savefig('lnA_vs_lgE.pdf')